# Pré-Processamento: SparkSQL

### Step 1: Start Spark Session

In [1]:
from pyspark.sql import SparkSession

try:
    spark.stop()
except NameError:
    print("SparkContext not defined")

# local mode
spark = SparkSession.builder \
            .appName("Spark SQL") \
            .master("local[*]") \
	    	.config("spark.jars.packages", "org.xerial.snappy:snappy-java:1.1.10.1") \
	    	.getOrCreate()

# cluster mode
#spark = SparkSession.builder \
#            .appName("Spark SQL basic example") \
#            .master("spark://spark:7077") \
#	    	.config("spark.some.config.option", "some-value") \
#	    	.getOrCreate()

SparkContext not defined


:: loading settings :: url = jar:file:/app/.venv/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2.5.2/cache
The jars for the packages stored in: /root/.ivy2.5.2/jars
org.xerial.snappy#snappy-java added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b6d0f42a-5364-4bff-b07d-fd9fce39de84;1.0
	confs: [default]
	found org.xerial.snappy#snappy-java;1.1.10.1 in central
downloading https://repo1.maven.org/maven2/org/xerial/snappy/snappy-java/1.1.10.1/snappy-java-1.1.10.1.jar ...
	[SUCCESSFUL ] org.xerial.snappy#snappy-java;1.1.10.1!snappy-java.jar(bundle) (1138ms)
:: resolution report :: resolve 1597ms :: artifacts dl 1142ms
	:: modules in use:
	org.xerial.snappy#snappy-java;1.1.10.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       |

In [3]:
from pathlib import Path
import gdown
import zipfile

ROOT_DIR = Path.cwd().parent
data_path = ROOT_DIR / "data"
data_raw_path = data_path / "data_raw"
zip_path = data_path / "data_raw.zip"
file_id = "1wLFeP8SPEuq_Ac6cPZTFE4PlR3bYdMRP"
download_url = f"https://drive.google.com/uc?export=download&id={file_id}"

if not data_path.exists() or not data_raw_path.exists():
    print(f"Pasta {data_raw_path} não encontrada. Baixando os dados...")
    
    data_path.mkdir(parents=True, exist_ok=True)

    gdown.download(download_url, str(zip_path), quiet=False)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(data_path)

    zip_path.unlink()

    print("Download e extração concluídos.")
else:
    print(f"Pasta {data_raw_path} encontrada.")

Pasta /app/data/data_raw não encontrada. Baixando os dados...


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1wLFeP8SPEuq_Ac6cPZTFE4PlR3bYdMRP
From (redirected): https://drive.google.com/uc?export=download&id=1wLFeP8SPEuq_Ac6cPZTFE4PlR3bYdMRP&confirm=t&uuid=56c8f1bd-8f35-4098-8c13-3ce93da9fc9c
To: /app/data/data_raw.zip
100%|██████████| 1.41G/1.41G [04:51<00:00, 4.84MB/s]


Download e extração concluídos.


In [4]:
import glob

arquivos_detalhados = glob.glob("../data/data_raw/**/*_DETAIL_*.csv", recursive=True)

arquivos_principais = [
    arq for arq in glob.glob("../data/data_raw/**/*.csv", recursive=True)
    if "_DETAIL_" not in arq
]

In [ ]:
arquivos_detalhados = "../datasample/RESTRICAO_COFF_EOLICA_DETAIL_2025_05_amostra.csv"

arquivos_principais = "../datasample/RESTRICAO_COFF_EOLICA_2025_05_amostra.csv"

In [5]:
df_detalhado = spark.read.option("delimiter", ";").option("header", True).option("inferSchema", True).csv(arquivos_detalhados)

df_principal = spark.read.option("delimiter", ";").option("header", True).option("inferSchema", True).csv(arquivos_principais)

In [6]:
df_detalhado.show()
df_detalhado.count()

+-------------+---------+----------------------+--------------------+------------+------+--------------------+-------------------+-------------------+---------------------+-------------------+---------------------+
|id_subsistema|id_estado|nom_modalidadeoperacao|   nom_conjuntousina|   nom_usina|id_ons|                 ceg|       din_instante|val_ventoverificado|flg_dadoventoinvalido|val_geracaoestimada|val_geracaoverificada|
+-------------+---------+----------------------+--------------------+------------+------+--------------------+-------------------+-------------------+---------------------+-------------------+---------------------+
|            N|       MA|             Tipo II-C| Conj. Paulino Neves|   Delta 3 I|MAEDT1|EOL.CV.MA.033682-...|2023-09-01 00:00:00|              9.249|                  0.0|             25.291|                23.24|
|            N|       MA|             Tipo II-C| Conj. Paulino Neves|  Delta 3 II|MAEDT2|EOL.CV.MA.033683-...|2023-09-01 00:00:00|          

55577477

In [7]:
df_principal.show()
df_principal.count()

+-------------+--------------+---------+----------+-------------------+---------+---+-------------------+-----------+-------------------+-------------------+---------------------+--------------------------+------------------+-------------------+
|id_subsistema|nom_subsistema|id_estado|nom_estado|          nom_usina|   id_ons|ceg|       din_instante|val_geracao|val_geracaolimitada|val_disponibilidade|val_geracaoreferencia|val_geracaoreferenciafinal|cod_razaorestricao|cod_origemrestricao|
+-------------+--------------+---------+----------+-------------------+---------+---+-------------------+-----------+-------------------+-------------------+---------------------+--------------------------+------------------+-------------------+
|            N|         NORTE|       MA|  MARANHAO|CONJ. PAULINO NEVES|CJU_MAPLN|  -|2024-08-01 00:00:00|    156.526|               NULL|              396.4|              169.492|                      NULL|              NULL|               NULL|
|            N| 

9503712

In [10]:
df_detalhado.write.option("compression", "uncompressed").mode("overwrite").parquet("../data/data_cleared/dados_detalhados.parquet")

In [9]:
df_principal.write.option("compression", "uncompressed").mode("overwrite").parquet("../data/data_cleared/dados_principais.parquet")